In [ ]:
# Import required libraries
import sys
import os
from pathlib import Path

# Add src directory to Python path
sys.path.append(str(Path.cwd().parent / 'src'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set up plotting
plt.style.use('default')
sns.set_palette('husl')
%matplotlib inline

print("📦 Libraries imported successfully!")
print(f"🐼 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")

## 📥 Load Data

Load the data that was processed in the data ingestion notebook.

In [ ]:
# Load the processed data
data_path = Path('../data/processed/loaded_data.csv')

if data_path.exists():
    df = pd.read_csv(data_path)
    print(f"✅ Data loaded from: {data_path}")
    print(f"📊 Shape: {df.shape}")
else:
    # Fallback: create demo data if no processed data exists
    print("⚠️ No processed data found. Creating demo data...")
    print("💡 Run the data ingestion notebook first for your own data")
    
    # Import our data connector
    try:
        from data_connector import load_demo_data
        df = load_demo_data()
        print(f"✅ Demo data created: {df.shape}")
    except ImportError:
        # Create basic demo data if import fails
        dates = pd.date_range('2024-01-01', periods=100)
        df = pd.DataFrame({
            'Date': dates,
            'Category': np.random.choice(['A', 'B', 'C'], 100),
            'Value': np.random.randint(100, 1000, 100),
            'Region': np.random.choice(['North', 'South'], 100)
        })
        print(f"✅ Basic demo data created: {df.shape}")

# Display basic info
print(f"\n📋 Columns: {list(df.columns)}")
df.head()

## 🔍 Data Exploration

Let's start with basic data exploration and understanding.

In [ ]:
# Basic dataset information
print("📊 DATASET OVERVIEW")
print("=" * 50)
print(f"📏 Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"💾 Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"📅 Data loaded at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print("\n📋 Column Information:")
print("-" * 30)
for i, (col, dtype) in enumerate(zip(df.columns, df.dtypes), 1):
    null_count = df[col].isnull().sum()
    null_pct = (null_count / len(df)) * 100
    print(f"{i:2d}. {col:<20} | {str(dtype):<10} | {null_count:3d} nulls ({null_pct:4.1f}%)")

In [ ]:
# Data types and statistical summary
print("📈 STATISTICAL SUMMARY")
print("=" * 50)

# Numerical columns
numerical_cols = df.select_dtypes(include=[np.number]).columns
if len(numerical_cols) > 0:
    print(f"🔢 Numerical columns ({len(numerical_cols)}): {list(numerical_cols)}")
    display(df[numerical_cols].describe())
else:
    print("🔢 No numerical columns found")

print("\n" + "=" * 50)

# Categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns
if len(categorical_cols) > 0:
    print(f"🏷️ Categorical columns ({len(categorical_cols)}): {list(categorical_cols)}")
    
    for col in categorical_cols:
        unique_count = df[col].nunique()
        print(f"\n📊 {col}:")
        print(f"   Unique values: {unique_count}")
        
        if unique_count <= 10:  # Show values if not too many
            value_counts = df[col].value_counts().head(10)
            for value, count in value_counts.items():
                pct = (count / len(df)) * 100
                print(f"   - {value}: {count:,} ({pct:.1f}%)")
        else:
            print(f"   Top values: {df[col].value_counts().head(3).index.tolist()}")
else:
    print("🏷️ No categorical columns found")

## 🧹 Data Quality Assessment

Check for data quality issues like missing values, duplicates, and outliers.

In [ ]:
print("🔍 DATA QUALITY ASSESSMENT")
print("=" * 50)

# Missing values analysis
missing_data = df.isnull().sum().sort_values(ascending=False)
missing_data = missing_data[missing_data > 0]

if len(missing_data) > 0:
    print("❌ Missing Values Found:")
    for col, count in missing_data.items():
        pct = (count / len(df)) * 100
        print(f"   {col}: {count:,} ({pct:.1f}%)")
    
    # Create missing data heatmap if we have missing values
    plt.figure(figsize=(10, 6))
    sns.heatmap(df.isnull(), cbar=True, yticklabels=False, cmap='viridis')
    plt.title('Missing Values Heatmap')
    plt.xlabel('Columns')
    plt.ylabel('Rows')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("✅ No missing values found!")

# Duplicate rows
duplicate_rows = df.duplicated().sum()
if duplicate_rows > 0:
    print(f"\n❌ Duplicate rows: {duplicate_rows:,} ({(duplicate_rows/len(df)*100):.1f}%)")
else:
    print("\n✅ No duplicate rows found!")

# Data type consistency
print("\n📋 Data Type Analysis:")
print("-" * 30)

# Check for date columns that might need conversion
potential_date_cols = []
for col in df.select_dtypes(include=['object']).columns:
    if 'date' in col.lower() or 'time' in col.lower():
        potential_date_cols.append(col)
        
if potential_date_cols:
    print(f"📅 Potential date columns: {potential_date_cols}")
    print("💡 Consider converting these to datetime format")

# Check for numerical columns stored as text
potential_numeric_cols = []
for col in df.select_dtypes(include=['object']).columns:
    sample_values = df[col].dropna().head(10)
    numeric_count = 0
    for val in sample_values:
        try:
            float(str(val).replace(',', '').replace('$', '').replace('%', ''))
            numeric_count += 1
        except:
            pass
    if numeric_count >= 7:  # If most values look numeric
        potential_numeric_cols.append(col)
        
if potential_numeric_cols:
    print(f"🔢 Potential numeric columns stored as text: {potential_numeric_cols}")
    print("💡 Consider converting these to numeric format")

## 📊 Descriptive Analytics

Perform descriptive analysis to understand patterns and distributions in your data.

In [ ]:
# Numerical analysis
if len(numerical_cols) > 0:
    print("📈 NUMERICAL DATA ANALYSIS")
    print("=" * 50)
    
    # Create distribution plots
    fig, axes = plt.subplots(2, min(len(numerical_cols), 3), figsize=(15, 8))
    if len(numerical_cols) == 1:
        axes = [axes]
    axes = axes.flatten() if len(numerical_cols) > 1 else axes
    
    for i, col in enumerate(numerical_cols[:6]):  # Max 6 plots
        if i < len(axes):
            # Distribution plot
            if i < 3:
                sns.histplot(data=df, x=col, kde=True, ax=axes[i])
                axes[i].set_title(f'Distribution: {col}')
            else:
                # Box plot for second row
                sns.boxplot(data=df, y=col, ax=axes[i])
                axes[i].set_title(f'Box Plot: {col}')
    
    # Hide empty subplots
    for i in range(len(numerical_cols), len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.show()
    
    # Correlation analysis if we have multiple numerical columns
    if len(numerical_cols) > 1:
        print("\n🔗 Correlation Analysis:")
        correlation_matrix = df[numerical_cols].corr()
        
        plt.figure(figsize=(10, 8))
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
                   square=True, fmt='.2f')
        plt.title('Correlation Matrix')
        plt.tight_layout()
        plt.show()
        
        # Highlight strong correlations
        strong_correlations = []
        for i in range(len(correlation_matrix.columns)):
            for j in range(i+1, len(correlation_matrix.columns)):
                corr = correlation_matrix.iloc[i, j]
                if abs(corr) > 0.7:  # Strong correlation threshold
                    strong_correlations.append((
                        correlation_matrix.columns[i], 
                        correlation_matrix.columns[j], 
                        corr
                    ))
        
        if strong_correlations:
            print("\n🎯 Strong Correlations (|r| > 0.7):")
            for col1, col2, corr in strong_correlations:
                print(f"   {col1} ↔ {col2}: {corr:.3f}")
        else:
            print("\n📊 No strong correlations found (|r| > 0.7)")

In [ ]:
# Categorical analysis
if len(categorical_cols) > 0:
    print("🏷️ CATEGORICAL DATA ANALYSIS")
    print("=" * 50)
    
    # Create count plots for categorical variables
    n_cats = min(len(categorical_cols), 4)  # Limit to 4 plots
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    axes = axes.flatten()
    
    for i, col in enumerate(categorical_cols[:4]):
        if df[col].nunique() <= 20:  # Only plot if not too many categories
            value_counts = df[col].value_counts().head(10)
            
            # Bar plot
            axes[i].bar(range(len(value_counts)), value_counts.values)
            axes[i].set_title(f'Count: {col}')
            axes[i].set_xticks(range(len(value_counts)))
            axes[i].set_xticklabels(value_counts.index, rotation=45, ha='right')
            
            # Add value labels on bars
            for j, v in enumerate(value_counts.values):
                axes[i].text(j, v + max(value_counts.values) * 0.01, str(v), 
                           ha='center', va='bottom')
        else:
            axes[i].text(0.5, 0.5, f'Too many categories\n({df[col].nunique()}) to display', 
                        ha='center', va='center', transform=axes[i].transAxes)
            axes[i].set_title(f'Categories: {col} ({df[col].nunique()})')
    
    # Hide empty subplots
    for i in range(len(categorical_cols), 4):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.show()
    
    # Cross-tabulation if we have multiple categorical columns
    if len(categorical_cols) >= 2:
        print("\n🔄 Cross-tabulation Analysis:")
        col1, col2 = categorical_cols[0], categorical_cols[1]
        
        if df[col1].nunique() <= 10 and df[col2].nunique() <= 10:
            crosstab = pd.crosstab(df[col1], df[col2], margins=True)
            print(f"\n📊 {col1} vs {col2}:")
            display(crosstab)
            
            # Heatmap of cross-tabulation
            plt.figure(figsize=(10, 6))
            sns.heatmap(crosstab.iloc[:-1, :-1], annot=True, fmt='d', cmap='Blues')
            plt.title(f'Cross-tabulation: {col1} vs {col2}')
            plt.tight_layout()
            plt.show()
        else:
            print(f"   {col1} vs {col2}: Too many categories for detailed cross-tab")

## 📅 Time Series Analysis (if applicable)

If your data has date/time columns, let's analyze trends over time.

In [ ]:
# Look for date columns
date_columns = []
for col in df.columns:
    if 'date' in col.lower() or 'time' in col.lower():
        date_columns.append(col)
    else:
        # Try to convert and see if it looks like dates
        try:
            sample = df[col].dropna().head(5)
            converted = pd.to_datetime(sample, errors='coerce')
            if not converted.isnull().all():
                date_columns.append(col)
        except:
            pass

if date_columns:
    print("📅 TIME SERIES ANALYSIS")
    print("=" * 50)
    print(f"Found potential date columns: {date_columns}")
    
    # Use the first date column
    date_col = date_columns[0]
    
    try:
        # Convert to datetime
        df_time = df.copy()
        df_time[date_col] = pd.to_datetime(df_time[date_col])
        
        print(f"\n📊 Date range: {df_time[date_col].min()} to {df_time[date_col].max()}")
        print(f"📅 Duration: {(df_time[date_col].max() - df_time[date_col].min()).days} days")
        
        # Time series plots
        if len(numerical_cols) > 0:
            fig, axes = plt.subplots(min(len(numerical_cols), 2), 1, figsize=(12, 8))
            if len(numerical_cols) == 1:
                axes = [axes]
            
            for i, num_col in enumerate(numerical_cols[:2]):
                # Group by date and aggregate
                daily_agg = df_time.groupby(df_time[date_col].dt.date)[num_col].sum().reset_index()
                daily_agg[date_col] = pd.to_datetime(daily_agg[date_col])
                
                axes[i].plot(daily_agg[date_col], daily_agg[num_col], marker='o', markersize=3)
                axes[i].set_title(f'Time Series: {num_col}')
                axes[i].set_xlabel('Date')
                axes[i].set_ylabel(num_col)
                axes[i].tick_params(axis='x', rotation=45)
            
            plt.tight_layout()
            plt.show()
            
            # Basic trend analysis
            print("\n📈 Trend Analysis:")
            for num_col in numerical_cols[:3]:  # Analyze first 3 numeric columns
                daily_agg = df_time.groupby(df_time[date_col].dt.date)[num_col].sum()
                
                # Simple trend calculation
                if len(daily_agg) > 1:
                    trend = daily_agg.iloc[-1] - daily_agg.iloc[0]
                    trend_pct = (trend / daily_agg.iloc[0]) * 100 if daily_agg.iloc[0] != 0 else 0
                    
                    direction = "📈" if trend > 0 else "📉" if trend < 0 else "📊"
                    print(f"   {direction} {num_col}: {trend:+.2f} ({trend_pct:+.1f}%) from start to end")
        
    except Exception as e:
        print(f"❌ Error in time series analysis: {e}")
        print("💡 Check if the date column format is correct")
        
else:
    print("📅 No date/time columns detected for time series analysis")
    print("💡 If you have date data, ensure column names contain 'date' or 'time'")

## 🎯 Key Insights Summary

Let's summarize the key findings from our analysis.

In [ ]:
print("🎯 KEY INSIGHTS SUMMARY")
print("=" * 60)

insights = []

# Dataset overview
insights.append(f"📊 Dataset contains {df.shape[0]:,} rows and {df.shape[1]} columns")

# Data quality insights
missing_pct = (df.isnull().sum().sum() / (df.shape[0] * df.shape[1])) * 100
if missing_pct > 0:
    insights.append(f"❌ Data completeness: {100-missing_pct:.1f}% ({missing_pct:.1f}% missing values)")
else:
    insights.append("✅ Data is complete with no missing values")

# Duplicate check
duplicate_pct = (df.duplicated().sum() / len(df)) * 100
if duplicate_pct > 0:
    insights.append(f"❌ Found {duplicate_pct:.1f}% duplicate rows")
else:
    insights.append("✅ No duplicate rows detected")

# Numerical insights
if len(numerical_cols) > 0:
    insights.append(f"📈 {len(numerical_cols)} numerical columns available for quantitative analysis")
    
    # Find columns with high variability
    high_var_cols = []
    for col in numerical_cols:
        cv = df[col].std() / df[col].mean() if df[col].mean() != 0 else 0
        if cv > 1:  # Coefficient of variation > 1
            high_var_cols.append(col)
    
    if high_var_cols:
        insights.append(f"📊 High variability detected in: {', '.join(high_var_cols)}")

# Categorical insights
if len(categorical_cols) > 0:
    insights.append(f"🏷️ {len(categorical_cols)} categorical columns for segmentation analysis")
    
    # Find columns with many categories
    high_card_cols = [col for col in categorical_cols if df[col].nunique() > 20]
    if high_card_cols:
        insights.append(f"🔍 High cardinality columns (>20 categories): {', '.join(high_card_cols)}")

# Date insights
if date_columns:
    insights.append(f"📅 Time series analysis available with {len(date_columns)} date column(s)")

# Print all insights
for i, insight in enumerate(insights, 1):
    print(f"{i:2d}. {insight}")

print("\n" + "=" * 60)
print("🚀 NEXT STEPS RECOMMENDATIONS:")
print("-" * 30)

recommendations = []

if missing_pct > 5:
    recommendations.append("🧹 Consider data cleaning strategies for missing values")

if len(numerical_cols) >= 2:
    recommendations.append("📊 Explore relationships between numerical variables")

if len(categorical_cols) >= 1 and len(numerical_cols) >= 1:
    recommendations.append("🔍 Perform segmentation analysis by categories")

if date_columns:
    recommendations.append("📈 Conduct detailed time series analysis and forecasting")

recommendations.extend([
    "📋 Create detailed visualizations in the next notebook",
    "💾 Save processed insights for reporting",
    "🔄 Set up automated analysis pipeline"
])

for i, rec in enumerate(recommendations, 1):
    print(f"{i}. {rec}")

## 💾 Export Analysis Results

Save your analysis results for further use and reporting.

In [ ]:
# Create analysis summary
analysis_summary = {
    'analysis_date': datetime.now().isoformat(),
    'dataset_info': {
        'shape': df.shape,
        'columns': list(df.columns),
        'memory_usage_mb': df.memory_usage(deep=True).sum() / 1024**2
    },
    'data_quality': {
        'missing_values_pct': (df.isnull().sum().sum() / (df.shape[0] * df.shape[1])) * 100,
        'duplicate_rows': int(df.duplicated().sum()),
        'complete_rows': int((~df.isnull().any(axis=1)).sum())
    },
    'column_types': {
        'numerical': list(numerical_cols),
        'categorical': list(categorical_cols),
        'datetime': date_columns
    }
}

# Add statistical summaries
if len(numerical_cols) > 0:
    analysis_summary['numerical_summary'] = df[numerical_cols].describe().to_dict()

if len(categorical_cols) > 0:
    cat_summary = {}
    for col in categorical_cols:
        cat_summary[col] = {
            'unique_values': int(df[col].nunique()),
            'top_values': df[col].value_counts().head(5).to_dict()
        }
    analysis_summary['categorical_summary'] = cat_summary

# Save summary as JSON
import json
summary_path = Path('../data/exports/analysis_summary.json')
summary_path.parent.mkdir(parents=True, exist_ok=True)

with open(summary_path, 'w') as f:
    json.dump(analysis_summary, f, indent=2, default=str)

print(f"💾 Analysis summary saved to: {summary_path}")

# Save processed data with any cleaning
clean_data_path = Path('../data/processed/analyzed_data.csv')
df.to_csv(clean_data_path, index=False)
print(f"💾 Analyzed data saved to: {clean_data_path}")

print("\n✅ Analysis complete! Ready for visualization and deeper insights.")
print("🎯 Next: Open the visualization notebook to create charts and graphs.")